**Bài tập cá nhân:** Tập tin Khảo sát món ăn sáng và trưa ưa thích ghi nhận lại các món ăn sáng và trưa ưa thích của các sinh viên với giả định sinh viên sẽ phải dùng bữa sáng và trưa mua tại các xe/tiệm ăn/nhà hàng (có thể là ăn tại chỗ hoặc mua mang đi) và cũng không bị hạn chế tài chính. Mỗi sinh viên được đề nghị tối thiểu 5 món ăn và tối đa 10 món mà họ ưa thích nhất. Khi liệt kê tên món ăn: có thể ghi món ăn chi tiết hơn 1 chút nếu muốn
*Ví dụ:* Bánh mì chả cá (thay vì chỉ ghi Bánh mì), Cơm tấm sườn bì chả (thay vì chỉ ghi chung Cơm tấm), Phớ tái nạm (thay vì chỉ ghi chung là Phở bò),...

Một danh sách mà sinh viên  ghi ra có thể như sau

Bánh mì bì, Cơm tấm, Hủ tíu Nam Vang, Bún riêu, Cơm Gà, Cơm cuộn, Hamburger

**Yêu cầu:**
Anh chị tải vể bảng khảo sát ăn sáng và trưa. Anh chị cần tiền xử lý thông tin trong bảng khảo sát này để có thể sử dụng luật kết hợp nhằm tìm ra xem có các sự liên hệ giữa các món ăn hay không. 
Yêu cầu: Tìm sự liên hệ (nếu có) chỉ với các món ăn: Cơm (tất cả các loại), Phở (tất cả các loại), Hủ tíu/Mì (tất cả các loại), Bún(tất cả các loại) và Bánh mì (tất cả các loại)
Lưu ý: Anh chị toàn quyền quyết định việc chuẩn hóa dữ liệu, giá trị minSup và giá trị minConf.



In [12]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [13]:
file_path = 'KhaoSatMonAn.xlsx' 

try:
    # header=2 vì tiêu đề nằm ở dòng thứ 3 (index 2)
    df_khaosat = pd.read_excel(file_path, sheet_name='KhaoSatMonAn', header=2, engine='openpyxl')
    print("--- Đã đọc sheet KhaoSatMonAn ---")
    print("Các cột tìm thấy:", df_khaosat.columns.tolist())

    # Đọc Sheet Tiền Xử Lý
    df_tienxuly = pd.read_excel(file_path, sheet_name='TienXuLyDuLieu', header=1, engine='openpyxl')
    print("--- Đã đọc sheet TienXuLyDuLieu ---")
    
    # Tìm cột có chữ "Danh sách món ăn"
    col_target = [c for c in df_khaosat.columns if "Danh sách món ăn" in str(c)]
    
    if len(col_target) > 0:
        raw_data = df_khaosat[col_target[0]].dropna()
        print(f"\n=> Đã lấy được dữ liệu từ cột: '{col_target[0]}'")
        print(f"=> Tổng số dòng: {len(raw_data)}")
    else:
        print("\nLỖI: Không tìm thấy cột nào tên là 'Danh sách món ăn...'")
        raw_data = []

except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file '{file_path}'. Bạn kiểm tra lại tên file nhé.")
except Exception as e:
    print(f"LỖI KHÁC: {e}")



--- Đã đọc sheet KhaoSatMonAn ---
Các cột tìm thấy: ['Máy', 'Danh sách món ăn anh chị chọn lựa (tối thiểu 5 tối đa 10)']
--- Đã đọc sheet TienXuLyDuLieu ---

=> Đã lấy được dữ liệu từ cột: 'Danh sách món ăn anh chị chọn lựa (tối thiểu 5 tối đa 10)'
=> Tổng số dòng: 76


In [ ]:
if 'raw_data' in locals() and len(raw_data) > 0:
    
    # Hàm chuẩn hóa (Dựa trên yêu cầu Cơm/Phở/Bún/Mì/Bánh mì)
    def normalize_food(food_item):
        item = str(food_item).lower().strip()
        if 'cơm' in item or 'com' in item: return 'Cơm'
        if 'phở' in item or 'pho' in item: return 'Phở'
        if any(x in item for x in ['bánh mì', 'bánh mỳ', 'banh mi']): return 'Bánh mì'
        if 'bún' in item or 'bun' in item: return 'Bún'
        if any(x in item for x in ['mì', 'mi', 'hủ tíu', 'hủ tiếu']): return 'Hủ tíu/Mì'
        return None

    # Tạo transactions
    transactions = []
    for response in raw_data:
        items = str(response).split(',')
        clean_transaction = set()
        for item in items:
            mapped_item = normalize_food(item)
            if mapped_item:
                clean_transaction.add(mapped_item)
        
        if len(clean_transaction) > 0:
            transactions.append(list(clean_transaction))

    # Chạy Apriori
    print(f"\nSố lượng giao dịch sạch để chạy thuật toán: {len(transactions)}")
    
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df_trans = pd.DataFrame(te_ary, columns=te.columns_)

    # min_support = 0.15 (15%)
    frequent_itemsets = apriori(df_trans, min_support=0.15, use_colnames=True)
    
    # min_confidence = 0.3 (30%)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
    
    # Hiển thị kết quả
    if not rules.empty:
        result = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values('lift', ascending=False)
        print("\n--- KẾT QUẢ ---")
        display(result)
    else:
        print("Không tìm thấy luật nào thỏa mãn điều kiện min_support/min_confidence.")